## 5. Algorithmic Questions (AQ)

### Part A
A sports club hires you to create a team for the National Sports Championship. Every Italian Region sends its best $M$ athletes to compete in an intense 2-day sports event, and Rome is no exception!

The trainers of Team Rome need to carefully choose the best $M$ athletes from a pool of $N$ candidates. Each athlete is uniquely identified by a number from 1 to $N$ and possesses a set of $S$ sports skills. Each skill is represented by a 3-character string with only uppercase letters and a non-negative integer indicating the athlete's proficiency in that skill (always greater than 0).

The trainers have extensively studied the competition format and established an optimal set of (possibly repeated) skills the team should possess to ensure the best possible performance. Each of the ten selected athletes will be assigned one of these skills as their role within the team.

The team's overall score is the sum of the skill scores of its members in the roles they have been assigned. Other skills of each athlete do not contribute to the team's score.

Your task is to determine the maximum possible global score for Team Rome, given the list of candidates.

Note: Assigning an athlete to a role not listed in their skills is possible. In that case, that athlete's contribution to the global score will be 0.

__Input__
The input consists of $2 + N(S + 1)$ lines:
- Line 1: the numbers $N, M,$ and $S$, separated by a space.
- Line 2: the optimal set of skills required by the trainers, as a list of $M$ space-separated skill names.
- Lines 3, . . . , $N(S + 1) + 2$: every group of $S + 1$ lines is formatted as follows:
  - Line 1: the unique id of the athlete.
  - Lines 2, . . . , $S + 1$: one skill name and the corresponding skill score, separated by a space.

__Output__
Print the maximum global score that can be achieved with the available athletes.

__Input 1__
```
14 10 1 # N, M, S
SWM VOL ATH VOL VOL BSK HCK BSK SWM BSK #set of skills
1
BSK 98
2
ATH 14
3
HCK 82
4
HCK 9
5
FTB 90
6
ATH 52
7
HCK 95
8
TEN 85
9
RGB 46
10
SWM 16
11
VOL 32
12
SOC 41
13
SWM 59
14
SWM 34
```
__Output 1__
```
370
```
---
__Input 2__
```
14 10 2 # N, M, S
SWM VOL ATH VOL VOL BSK HCK BSK SWM BSK #set of skills
1
BSK 98
HCK 12
2
ATH 14
VOL 1
3
HCK 82
ATH 30
4
HCK 9
SWM 27
5
FTB 90
HCK 50
6
ATH 52
RGB 80
7
HCK 95
SWM 11
8
TEN 85
RGB 7
9
RGB 46
SWM 30
10
SWM 16
BSK 12
11
VOL 32
HCK 40
12
SOC 41
FTB 12
13
SWM 59
TEN 82
14
SWM 34
VOL 20
```
__Output 2__
```
399
```
__Your job__:
1. Implement an algorithm to solve the described mentioned problem. 

2. What is the __time complexity__ (the Big O notation) of your solution? Please provide a <ins>detailed explanation</ins> of how you calculated the time complexity.

3. Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. If the <ins>two differ</ins>, which one is right? (why?)
   
4. If you algorithm has exponential time complexity, can you provide a __polynomial-time version__? 

5. If $S=1$, how does the __time complexity__ of an optimal algorithm to solve this problem change?

### Part B

The success of a project depends not only on the expertise of the people involved but also on how effectively they work together as a team. So this time, instead of focusing on who has the best skills, let's focus on finding a group of individuals who can function as a team to accomplish a specific task.

Given a set of skills $T$, our goal is to find a set of individuals $X' \subseteq X$ , such that every required skill in $T$ is exhibited by at least one individual in $X'$. Additionally, the members of team $X'$ should have low effort to work together i.e. all the members of the team $X'$ work well with each other.

This problem can be easily visualised with graphs: we define an undirected weighted graph $G=(V,E)$ where every element $x_i \in X$ has a corresponding node $v_i \in V$. The weights of the edges represent the effort required to work well together: the lower the weight of an edge between two nodes, the less effort the corresponding team members need to work well together.

We define as acceptable solution any subset $V' \subseteq V$ such that $T \cap \cup_{v_i\in V'} S_{v_i}$ where $S_{v_i} =$ {set of skills of member $x_i$ corresponding to the vertex $v_i$}. The goal is to find, among all acceptable solutions, the one that minimizes the effort to work together $E_c(V')$.
The effort to work together $E_c(V')$ is the cost of the minimum spanning tree on the subgraph $G[V']$ i.e. the sum of the weights of its edges.

__Your job__:
1. Prove or disprove that the problem is NP-complete.
2. Write a heuristic in order to approximate the best solution for this problem.
3. What is the time complexity of your solution ?

### Part A

#### 1. Implementing an algorithm

Before starting the algorithm implementation, we need to parse the input. From the task description we get how the input is processed. Below is the implementation of parsing.

In [14]:
from collections import defaultdict

def parse_input(input_filename):
    with open(input_filename) as f:
        lines = f.readlines()

    # First line is n, m and s values respectively
    n, m, s = [int(x) for x in lines[0].split()]

    # Second line is set of skills needed for competition 
    list_of_skills = lines[1].split()

    athlete_dict = defaultdict(dict)
    i = 2

    # Rest of the lines is athlete's and their skills with scores 
    while i < len(lines):
        # If only id is present, that is the new athlete
        if len(lines[i].split()) == 1:
            athlete_id = int(lines[i].strip())
        # This is the skill and corresponding score for the athlete_id 
        else:
            skill, score = lines[i].split()
            athlete_dict[athlete_id][skill] = int(score)
        i += 1
    return n, m, s, list_of_skills, athlete_dict


After parsing, now we can talk about the algorithm. First idea is to implement DFS algorithm that will get through each and every athlete with every possible combination. This will give us an answer that is always correct, with the expense of time complexity, which we will talk about in the next point.

In [34]:
def dfs(athlete_dict, list_of_skills, used_athletes, current_index, current_score):
    # We reached the end of skills needed
    if current_index == len(list_of_skills):
        return current_score

    max_score = 0

    for athlete, skills in athlete_dict.items():
        # Each athlete must be unique, so only those that are not in used_athletes are interesting for consideration
        if athlete not in used_athletes:
            # Add that athlete to the list
            used_athletes.add(athlete)
            # Because we will consider each athlete for each skill, we will take that athlete into consideration
            # If he does not have that skill, in that case, give the score 0
            skill_score = skills.get(list_of_skills[current_index], 0)
            # Recursive call with increased current_index and current_score
            score = dfs(
                athlete_dict,
                list_of_skills,
                used_athletes,
                current_index + 1,
                current_score + skill_score,
            )
            # Get the score that is maximum
            max_score = max(max_score, score)
            # Now remove the athlete from used_athletes, so we can check the new skill for this athlete
            used_athletes.remove(athlete)

    return max_score


# Wrapper that takes input the file where the problem is stored using the DFS approach
def give_maximum_score_dfs(input_filename):
    n, m, s, list_of_skills, athlete_dict = parse_input(input_filename)
    used_athletes = set()
    return dfs(athlete_dict, list_of_skills, used_athletes, 0, 0)

In [31]:
max_score_1 = give_maximum_score_dfs("input/ex1.txt")
max_score_2 = give_maximum_score_dfs("input/ex2.txt")

In [33]:
print(max_score_1) # Expected: 370
print(max_score_2) # Expected: 399

370
399


As we can see, the results from the examples match our results so the algorithm is correct. More examples would be needed for further testing, but with this strategy we will always get correct result.

#### 2. Time complexity

For the time complexity, we will only focus on the algorithm, not on parsing the input. If we take a look at the algorithm, it will go through each athlete and their skills with `for athlete, skills in athlete_dict.items():`, which basically needs to be done `n` times, where `n` is the number of athletes. Inside the for loop all of the operations are `O(1)`, except for the recursive call `score = dfs(parameters)`. Now we go through this call again for `n`, and we do that times how many skills there are, which is `m`. In conclusion, the time complexity is `O(n^m)` where `m` is the total skills needed and n is the number of athletes.

As we can see, this is not the most optimal solution, however it provides the correct output everytime. To improve on this, some sort of heuristic or different approach is needed.

#### 3. ChatGPT time complexity

After asking ChatGPT for the time complexity of the code, following solution is provided: 

"The time complexity of the provided Depth-First Search (DFS) algorithm can be analyzed based on the number of athletes and the number of skills. Let's denote the number of athletes as N and the number of skills as M.
- Outer For Loop: The loop iterates over each athlete. Since each athlete can be considered once for each recursive call of dfs, this contributes to N iterations.
- Recursive Calls: The recursive call is made for each skill, incrementing current_index each time until it reaches the length of list_of_skills, which is M. This means the recursion depth is M.
- Combination of Athletes and Skills: In the worst case, every athlete is considered for every skill. The recursive calls explore every possible combination of athletes and skills.

Therefore, the time complexity of the dfs function can be approximated as O(N^M). This represents the scenario where for each skill (M levels deep in recursion), the algorithm considers each of the N athletes."

This gives the same answer as we did analyzied in point `2. Time complexity`. 

#### 4. Polynomial-time algorithm

Now, we will try to implement an algorithm that solves the problem in polynomial-time. The approach that will be taken is greedy. We will take the original athlete dictionary which contained athlete_id and skills with scores, create a new dictionary called skills dictionary. That dictionary will then be sorted with the highest values on top. So when we will need someone with the best score in that skill, we will take them as our team member.

In [35]:
def give_maximum_score_greedy(input_filename):
    n, m, s, list_of_skills, athlete_dict = parse_input(input_filename)
    skill_dict = defaultdict(list)

    # Creation of skills dictionary
    for athlete_id, skills in athlete_dict.items():
        for skill, score in skills.items():
            skill_dict[skill].append((score, athlete_id))

    # Making the athletes with highest scores be on top
    for skill in skill_dict:
        skill_dict[skill].sort(reverse=True)


    assigned_athletes = set()
    total_score = 0

    # Now we go through all the skills and take the athletes with the highest score
    for skill in list_of_skills:
        for score, athlete_id in skill_dict[skill]:
            # If they are assigned already, don't take them into consideration
            if athlete_id not in assigned_athletes:
                total_score += score
                assigned_athletes.add(athlete_id)
                break

    return total_score


In [36]:
max_score_1 = give_maximum_score_greedy("input/ex1.txt")
max_score_2 = give_maximum_score_greedy("input/ex2.txt")

In [38]:
print(max_score_1) # Expected: 370
print(max_score_2) # Expected: 399

370
399


We got the same results as with our previous approach, so it is correct. However, this approach might not always give us correct results. Depending on the input, there is still possibility that someone gets assigned a skill (eg. basketball) that individually he is the best in, but he is also best in some other skill (eg. football), so having that some solutions are not the most optimal for the total team score. 

For the time complexity, most of the time is spent on creating and sorting the new dictionary for skills. which can be seen as `O(S x N log N)`, where `S` is number of the skills athlete can possess, and `N` is the number of athletes. Depending on the size of skills, this can be reduced to `O(N log N)`, if the number of athletes is much larger than number of skills. Which in the end both solutions are in polynomial time. 

#### 4. S = 1?

As discussed in the previous paragraph, if `S = 1` then the greedy approach will only be expensive on sorting which is `O(N log N)`. However, if we look at the initial DFS approach, it does not matter the `S` value. It is only dependent on the N and M, which has also been discussed in previous points.